# Copolymerization

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/HugoMVale/polykin/main?labpath=docs%2Ftutorials%2Fcopolymerization.ipynb)

## Overview

The `copolymerization` module offers a number of ...